In [ ]:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/14JY1DjWv6MaPmoSmyihDWRcVRkiTqmJO?usp=sharing)

SyntaxError: ignored

In [3]:
!pip install transformers

Finetuning

In [4]:
pip install simpletransformers tqdm pandas

In [32]:
import pandas as pd
train_df_prev = pd.read_csv("train.csv")
dev_df_prev = pd.read_csv("dev.csv")

In [33]:
train_df_prev.head(5)

,Unnamed: 0,topic,argument,key_point,stance,label
0,10137,We should ban human cloning,a person created through cloning could potenti...,Cloning is not understood enough yet,1,0
1,10138,We should ban human cloning,a person created through cloning could potenti...,Cloning is unethical/anti-religious,1,0
2,10139,We should ban human cloning,a person created through cloning could potenti...,Cloning is unnatural,1,0
3,10140,We should ban human cloning,a person created through cloning could potenti...,Cloning is unsafe,1,1
4,10141,We should ban human cloning,Allowing humans to be cloned is setting the wo...,Cloning is unethical/anti-religious,1,0


In [34]:
train_df_prev.columns

Index(['Unnamed: 0', 'topic', 'argument', 'key_point', 'stance', 'label'], dtype='object')

In [35]:
train_df = train_df_prev.filter(['argument', 'key_point', 'label'], axis=1)
dev_df = dev_df_prev.filter(['argument', 'key_point', 'label'], axis=1)

In [36]:
train_df.head(5)

,argument,key_point,label
0,a person created through cloning could potenti...,Cloning is not understood enough yet,0
1,a person created through cloning could potenti...,Cloning is unethical/anti-religious,0
2,a person created through cloning could potenti...,Cloning is unnatural,0
3,a person created through cloning could potenti...,Cloning is unsafe,1
4,Allowing humans to be cloned is setting the wo...,Cloning is unethical/anti-religious,0


In [37]:
# No Prompt Engineering

# For Train set

for i in train_df.index:
  arg = train_df['argument'][i]
  key = train_df['key_point'][i]
  if arg[-1] != '.':
    modified_arg = arg + '. ' + key + '.'
    train_df.at[i, 'argument'] = modified_arg
  else:
    modified_arg = arg + ' ' + key + '.'
    train_df.at[i, 'argument'] = modified_arg

# For Dev set

for i in dev_df.index:
  arg = dev_df['argument'][i]
  key = dev_df['key_point'][i]
  if arg[-1] != '.':
    modified_arg = arg + '. ' + key + '.'
    dev_df.at[i, 'argument'] = modified_arg
  else:
    modified_arg = arg + ' ' + key + '.'
    dev_df.at[i, 'argument'] = modified_arg

In [38]:
train_df.at[110, 'argument']

'human cloning allows experimentation which might lead to the development of a more robust strain of human therefore it should not be banned. Cloning can be used for organ replication.'

In [39]:
train_df = train_df.filter(['argument', 'label'], axis=1)
dev_df = dev_df.filter(['argument', 'label'], axis=1)

In [40]:
train_df["label"] = train_df["label"].apply(str)
dev_df["label"] = dev_df["label"].apply(str)

In [41]:
# Renaming the columns
train_df.columns = ["input_text","target_text"]
dev_df.columns = ["input_text","target_text"]

In [42]:
# Adding a prefix. Here we shall keep "paraphrase" as a prefix.
train_df["prefix"] = "text-classification"
dev_df["prefix"] = "text-classification"

In [43]:
from sklearn.model_selection import train_test_split
train_data, dev_data = train_df, dev_df

In [44]:
args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 256,
    "num_train_epochs": 4,
    "num_beams": None,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "use_multiprocessing": False,
    "save_steps": -1,
    "save_eval_checkpoints": True,
    "evaluate_during_training": False,
    "adam_epsilon": 1e-08,
    "eval_batch_size": 6,
    "fp_16": False,
    "gradient_accumulation_steps": 16,
    "learning_rate": 0.0003,
    "max_grad_norm": 1.0,
    "n_gpu": 1,
    "seed": 42,
    "train_batch_size": 6,
    "warmup_steps": 0,
    "weight_decay": 0.0,
}

In [45]:
from simpletransformers.t5 import T5Model
import sklearn
model = T5Model("t5","t5-small", args=args)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:169: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
model.train_model(train_data, eval_data=dev_data, use_cuda=True,acc=sklearn.metrics.accuracy_score)

  0%|          | 0/17019 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3538: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/2837 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


In [ ]:
from simpletransformers.t5 import T5Model
import os
root_dir = os.getcwd()
trained_model_path = os.path.join(root_dir,"outputs")
args = {
"overwrite_output_dir": True,
"max_seq_length": 256,
"max_length": 50,
"top_k": 50,
"top_p": 0.95,
"num_return_sequences": 5
}
trained_model = T5Model("t5",trained_model_path,args=args)

In [ ]:
prefix = "paraphrase"
pred = trained_model.predict([f"{prefix}: a libertarian system allows and ensures that a person is free to do as they please as long as those actions do not impede or infringe on the rights of others. This means "])
print(pred)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/5 [00:00<?, ?it/s]

[['Liberties are beneficial to those who are willing to act', 'Libertarian policies allow for change', 'It is within the freedoms of the parents to select the sex of the child', 'It is within the freedoms of the parents to select the sex of the child', 'People should have the freedom to choose to end their life']]


In [ ]:
print(test_data[0:1]['input_text'])

18304    we shouldnt legalize cannabis like any drug it...
Name: input_text, dtype: object


In [ ]:
from google.colab import files
files.download("/content/t5-model.zip")

In [ ]:
!zip -r /content/t5-model.zip /content/outputs

  adding: content/outputs/ (stored 0%)
  adding: content/outputs/checkpoint-675-epoch-3/ (stored 0%)
  adding: content/outputs/checkpoint-675-epoch-3/special_tokens_map.json (deflated 83%)
  adding: content/outputs/checkpoint-675-epoch-3/spiece.model (deflated 48%)
  adding: content/outputs/checkpoint-675-epoch-3/config.json (deflated 63%)
  adding: content/outputs/checkpoint-675-epoch-3/model_args.json (deflated 63%)
  adding: content/outputs/checkpoint-675-epoch-3/pytorch_model.bin (deflated 7%)
  adding: content/outputs/checkpoint-675-epoch-3/scheduler.pt (deflated 51%)
  adding: content/outputs/checkpoint-675-epoch-3/tokenizer_config.json (deflated 79%)
  adding: content/outputs/checkpoint-675-epoch-3/optimizer.pt (deflated 15%)
  adding: content/outputs/checkpoint-675-epoch-3/training_args.bin (deflated 49%)
  adding: content/outputs/special_tokens_map.json (deflated 83%)
  adding: content/outputs/spiece.model (deflated 48%)
  adding: content/outputs/checkpoint-450-epoch-2/ (store

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
